In [1]:
import psycopg2
import pandas as pd
import numpy as np

# Setup Database

In [2]:
def create_database():
    #connect to default db
    conn = psycopg2.connect("host = localhost dbname = postgres user= postgres password = root")
    conn.set_session(autocommit = True)
    cur = conn.cursor()
    
    #create a stock database
    
    cur.execute("DROP DATABASE stock")
    cur.execute("CREATE DATABASE stock")
    
    #close connection to default database
    conn.close()
    
    #connect to stock database
    
    conn = psycopg2.connect("host = localhost dbname = mystock user = postgres password = root")
    cur = conn.cursor()
    
    return cur, conn
    

In [3]:
def table_query(cur, conn):
    for query in queries:
        cur.execute(query)
        conn.commit()

In [4]:
cur, conn = create_database()

# Extract Data from Csv File

In [5]:
stock_data = pd.read_csv("Samsung.csv", index_col = False)

In [6]:
stock_data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [7]:
stock_data.shape

(5621, 7)

In [8]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5621 entries, 0 to 5620
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       5621 non-null   object 
 1   Open       5621 non-null   float64
 2   High       5621 non-null   float64
 3   Low        5621 non-null   float64
 4   Close      5621 non-null   float64
 5   Adj Close  5621 non-null   float64
 6   Volume     5621 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 307.5+ KB


In [9]:
stock_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2000-01-04,6000.0,6110.0,5660.0,6110.0,4651.737793,74195000
1,2000-01-05,5800.0,6060.0,5520.0,5580.0,4248.232422,74680000
2,2000-01-06,5750.0,5780.0,5580.0,5620.0,4278.686523,54390000
3,2000-01-07,5560.0,5670.0,5360.0,5540.0,4217.780273,40305000
4,2000-01-10,5600.0,5770.0,5580.0,5770.0,4392.884766,46880000


# Preprocessing Data

In [11]:
#giving each item an id so that we may refer to it as our primary key later
stock_data['id'] = np.arange(1,5622)
first_col = stock_data.pop('id')
stock_data.insert(0,'id',first_col)

In [14]:
#changing datetime object into datetime datatype
stock_data["Date"] = pd.to_datetime(stock_data['Date'])

In [15]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5621 entries, 0 to 5620
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         5621 non-null   int32         
 1   Date       5621 non-null   datetime64[ns]
 2   Open       5621 non-null   float64       
 3   High       5621 non-null   float64       
 4   Low        5621 non-null   float64       
 5   Close      5621 non-null   float64       
 6   Adj Close  5621 non-null   float64       
 7   Volume     5621 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int32(1), int64(1)
memory usage: 329.5 KB


In [17]:
stock_data.Date[4]

Timestamp('2000-01-10 00:00:00')

In [18]:
stock_data.head()

,id,Date,Open,High,Low,Close,Adj Close,Volume
0,1,2000-01-04,6000.0,6110.0,5660.0,6110.0,4651.737793,74195000
1,2,2000-01-05,5800.0,6060.0,5520.0,5580.0,4248.232422,74680000
2,3,2000-01-06,5750.0,5780.0,5580.0,5620.0,4278.686523,54390000
3,4,2000-01-07,5560.0,5670.0,5360.0,5540.0,4217.780273,40305000
4,5,2000-01-10,5600.0,5770.0,5580.0,5770.0,4392.884766,46880000


In [19]:
stock_data.dtypes

id                    int32
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object

# Load Data into Database

In [20]:
stocks_data_table = ("""CREATE TABLE IF NOT EXISTS stock_data( 
id int PRIMARY KEY,
Date timestamp,
Open float,
High float,
Low float,
Close float,
Adj_Close float,
Volume int);
""")

In [21]:
cur.execute(stocks_data_table)
conn.commit()

In [22]:
stocks_data_table_insert = """INSERT INTO stock_data (
id,
Date,
Open,
High,
Low,
Close,
Adj_Close,
Volume
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""

In [23]:
# Iterate over each row in the stock_data DataFrame using the iterrows() method
for i, row in stock_data.iterrows():
    
    # Execute the INSERT statement for each row, passing the values as a list
    # The list() method is used to convert the row (which is a Pandas Series) to a list
    cur.execute(stocks_data_table_insert, list(row))

In [24]:
conn.commit()

In [25]:
# Use a try-except block to catch any errors that might occur when selecting data from the table
try:
    cur.execute("SELECT * FROM stock_data;")
except psycopg2.Error as e:
    print("Error: select *")
    print(e)

# Fetch one row at a time from the result set and print it to the console
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, datetime.datetime(2000, 1, 4, 0, 0), 6000.0, 6110.0, 5660.0, 6110.0, 4651.737793, 74195000)
(2, datetime.datetime(2000, 1, 5, 0, 0), 5800.0, 6060.0, 5520.0, 5580.0, 4248.232422, 74680000)
(3, datetime.datetime(2000, 1, 6, 0, 0), 5750.0, 5780.0, 5580.0, 5620.0, 4278.686522999999, 54390000)
(4, datetime.datetime(2000, 1, 7, 0, 0), 5560.0, 5670.0, 5360.0, 5540.0, 4217.780272999999, 40305000)
(5, datetime.datetime(2000, 1, 10, 0, 0), 5600.0, 5770.0, 5580.0, 5770.0, 4392.884765999999, 46880000)
(6, datetime.datetime(2000, 1, 11, 0, 0), 5820.0, 6100.0, 5770.0, 5770.0, 4392.884765999999, 59745000)
(7, datetime.datetime(2000, 1, 12, 0, 0), 5610.0, 5740.0, 5600.0, 5720.0, 4354.818359000001, 29220000)
(8, datetime.datetime(2000, 1, 13, 0, 0), 5600.0, 5740.0, 5560.0, 5710.0, 4347.205078, 41190000)
(9, datetime.datetime(2000, 1, 14, 0, 0), 5720.0, 5880.0, 5680.0, 5830.0, 4438.565918, 49375000)
(10, datetime.datetime(2000, 1, 17, 0, 0), 6000.0, 6180.0, 5920.0, 6100.0, 4644.125, 63505000)
(11, d

(2356, datetime.datetime(2009, 3, 4, 0, 0), 9560.0, 9810.0, 9380.0, 9780.0, 7710.10791, 26067250)
(2357, datetime.datetime(2009, 3, 5, 0, 0), 9980.0, 10120.0, 9790.0, 10060.0, 7930.850586, 32982600)
(2358, datetime.datetime(2009, 3, 6, 0, 0), 9920.0, 10100.0, 9880.0, 10000.0, 7883.546387, 19412000)
(2359, datetime.datetime(2009, 3, 9, 0, 0), 10200.0, 10320.0, 9950.0, 10300.0, 8120.053223000001, 18225250)
(2360, datetime.datetime(2009, 3, 10, 0, 0), 10100.0, 10360.0, 10040.0, 10080.0, 7946.614745999999, 20458750)
(2361, datetime.datetime(2009, 3, 11, 0, 0), 10380.0, 10500.0, 10260.0, 10500.0, 8277.723633, 26584150)
(2362, datetime.datetime(2009, 3, 12, 0, 0), 10400.0, 10720.0, 10400.0, 10720.0, 8451.164063, 50741850)
(2363, datetime.datetime(2009, 3, 13, 0, 0), 10800.0, 11000.0, 10560.0, 10560.0, 8325.027344, 24128850)
(2364, datetime.datetime(2009, 3, 16, 0, 0), 10520.0, 10720.0, 10340.0, 10460.0, 8246.191406, 14711600)
(2365, datetime.datetime(2009, 3, 17, 0, 0), 10560.0, 10740.0, 103

(4479, datetime.datetime(2017, 9, 19, 0, 0), 52500.0, 52640.0, 51780.0, 52120.0, 45281.179688, 9850400)
(4480, datetime.datetime(2017, 9, 20, 0, 0), 52120.0, 52500.0, 51840.0, 52220.0, 45368.050781, 9292750)
(4481, datetime.datetime(2017, 9, 21, 0, 0), 52220.0, 52960.0, 52220.0, 52800.0, 45871.953125, 8839800)
(4482, datetime.datetime(2017, 9, 25, 0, 0), 53000.0, 53680.0, 53000.0, 53620.0, 46584.367188, 9303100)
(4483, datetime.datetime(2017, 9, 26, 0, 0), 53620.0, 53620.0, 53620.0, 53620.0, 46584.367188, 0)
(4484, datetime.datetime(2017, 9, 27, 0, 0), 52000.0, 52200.0, 51500.0, 51680.0, 44898.914063, 10797000)
(4485, datetime.datetime(2017, 9, 28, 0, 0), 52260.0, 52460.0, 51260.0, 51260.0, 44654.992188, 12025450)
(4486, datetime.datetime(2017, 9, 29, 0, 0), 51180.0, 51620.0, 50840.0, 51280.0, 44672.410156, 12909400)
(4487, datetime.datetime(2017, 10, 10, 0, 0), 53360.0, 53640.0, 52800.0, 52800.0, 45996.550781, 20519800)
(4488, datetime.datetime(2017, 10, 11, 0, 0), 53600.0, 54760.0, 5

In [ ]:
#cur.execute("rollback")

In [26]:
cur.close()
conn.close()